In [1]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 90.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.165
    Uninstalling botocore-1.29.165:
      Successfully uninstalled botocore-1.29.165
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.165
    Uninstalling boto3-1.26.165:
      Successfully uninstalled boto3-1.26.165
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.114 requires botocore==1.29.114, but you have botocore 1.31.30 which is incompatible.
awscli 1.27.114 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://p

In [ ]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'llm_english')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'llm_english')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

model_name = None
entry_point = 'inference.py'
framework_version = '1.13.1'
py_version = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = './code',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

endpoint_name = 'pytorch-inference-llm-baichuan13b'
instance_type = 'ml.g4dn.12xlarge'
instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

arn:aws:iam::150333911459:role/service-role/AmazonSageMaker-ExecutionRole-20220802T133420
sagemaker-us-west-2-150333911459
dummy
upload: ./model.tar.gz to s3://sagemaker-us-west-2-150333911459/llm_english/assets/model.tar.gz
-------

In [11]:
inputs= {
    "ask": "你好!"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "晚上睡不着应该怎么办"

}

response = predictor.predict(inputs)
print(response["answer"])

你好！有什么我能帮到你的吗？
如果晚上难以入睡，可以尝试以下方法：

1. 确保睡眠环境舒适：保持房间温度适中、光线柔和且安静。
2. 建立规律的作息时间：每天尽量在相同的时间上床睡觉和起床。
3. 避免咖啡因、尼古丁和酒精：这些物质可能影响你的睡眠质量。
4. 限制白天小憩：如果你白天打盹，可能会影响到晚上的睡眠。
5. 运动：定期进行有氧运动如散步或跑步有助于改善睡眠。但请注意不要在临近睡觉的时间段内做剧烈运动。
6. 放松身心：在睡前进行深呼吸练习、瑜伽或冥想等放松身心的活动。
7. 睡前不使用电子产品：手机、电脑等设备发出的蓝光会干扰人体生物钟，影响睡眠。
8. 避免过度饮食：尤其是在晚餐后，过重的饱腹感会影响睡眠。
9. 尝试渐进性肌肉松弛法：这是一种帮助你放松身体和减轻紧张感的技巧。
10. 如果长时间无法入睡，不要焦虑：离开卧室，进行一些轻松的活动，等到感到困意再回到床上。

如果以上方法都试过了还是无法改善失眠状况，建议寻求专业医生的帮助。
